In [1]:
from functions import *
from torch.optim import Adam
from functions_pt import *

In [2]:
embed_size = 160
dropout = 0.1
multiple_factor = 1
layers = 2
batch_size = 64
concat = True
agg = 'mean' # mean,sum,min,max
model = pointNetQ_rep(layers,embed_size,agg,dropout,multiple_factor,concat)#.to('cuda')
#model.load_state_dict(torch.load('baseline_noBN.pt'))
#model.eval()
model = model.to('cuda')

In [3]:
words_embed = np.load('possible_words_embed.npy')

RL Loop

In [4]:
# model trained from start
lr = 6e-5
clip = 0.03
epochs = 3
damp = 1
iterations = 30
max_data = 2000
states = set()
opt = Adam(model.parameters(),betas=(0.85, 0.95),lr=lr)
best_score = 100
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_saveQ(matrix,np.arange(2309),policy_modelQ_rep,2309,0,model=model,words_embed=words_embed,
                           top=0.6,eps=0.02/(i+1))
    print('iter:{}, score:{}'.format(i,score))
    if score < best_score:
        best_score = score
        best_weight = copy.deepcopy(model.state_dict()) 
    # prepare data for training
    index,val,log_prob,action,entro = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],device='cuda').to(torch.long)
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    entro = torch.tensor(entro,dtype=torch.float32,device='cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')
    action_rep = []
    for ids_,a_ in zip(index,action):
        for id_ in ids_:
            action_rep.append(matrix[a_,id_])
    action_rep = torch.tensor(np.array(action_rep),device='cuda').long()
    
    # policy update
    model.train()
    loss_int,counter_ = 1000,0
    tmpWeight = copy.deepcopy(model.state_dict())   
    while counter_ < epochs:
        yhat = model((words,length,action_rep,entro))
        loss = torch.sum(prob*(ys-yhat)**2)/torch.sum(prob)
        print('loss:{}'.format(loss.item()))
        if loss.item() > loss_int + 0.1:
            model.load_state_dict(tmpWeight)
            for g in opt.param_groups:
                g['lr'] = g['lr']/1.5
        else:
            loss_int = loss.item()
            counter_ += 1
            tmpWeight = copy.deepcopy(model.state_dict())
            loss.backward()
            clip_grad_value_(model.parameters(),clip)
            opt.step()
            opt.zero_grad()
    for g in opt.param_groups:
        g['lr'] = lr
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)
model.load_state_dict(best_weight)

iter:0, score:3.751840623646602
set size:428
loss:2.196314811706543
loss:1.4182347059249878
loss:0.9087357521057129
iter:1, score:3.8198354265915992
set size:859
loss:0.559843122959137
loss:0.37274736166000366
loss:0.30474889278411865
iter:2, score:3.803811173668255
set size:1292
loss:0.3642876148223877
loss:0.43358340859413147
loss:0.4097062051296234
iter:3, score:3.812472932005199
set size:1708
loss:0.6690727472305298
loss:0.5615126490592957
loss:0.2068507969379425
iter:4, score:3.8254655695106123
set size:2087
loss:0.24361346662044525
loss:0.26006144285202026
loss:0.2416672259569168
iter:5, score:3.787353832828066
set size:2470
loss:0.5064302086830139
loss:0.217472106218338
loss:0.2042892426252365
iter:6, score:3.7899523603291505
set size:2865
loss:0.18245336413383484
loss:0.16428406536579132
loss:0.4189923405647278
loss:0.4446525275707245
loss:0.4231463670730591
loss:0.41585108637809753
loss:0.4260367751121521
loss:0.3943471908569336
loss:0.6958982944488525
loss:0.16017092764377594

KeyboardInterrupt: 

In [5]:
torch.save(model.state_dict(), 'RL_Q0.pt')

In [5]:
evaluate(matrix,np.arange(2309),policy_modelQ_rep,2309,model=model,words_embed=words_embed,top=0.6,
                         eps=0)

3.8739714161974894